# 제어 아이디어2

## 가정
$T_{set}$은 실내기가 도달할 최종 온도 (도달하면 해당 실내기는 on/off 반복하며 온도 유지)를 결정하는 값이고 온도 변화 기울기에는 영향 x

-> 방전체의 목표온도 $Tgoal$이 정해지면 $T_{set} = T_{goal}-1$로 고정
# 1. 시스템 구성 요소 정리 (역할 분해)

## (1) 도달시간 예측 딥러닝 모델 $g(\cdot)$

각 실내기 $i=1,2,3$에 대해

$$
\hat t_i
= g_i\big(
\mathbf x_k,,
\mathbf u_k,,
\mathbf z_k
\big)
$$

* 상태
  $$
  \mathbf x_k = [T_{1,k}, T_{2,k}, T_{3,k}]^\top
  $$
* 입력(이산 풍량)
  $$
  \mathbf u_k = [u_{1,k}, u_{2,k}, u_{3,k}]^\top,\quad u_i\in{0,1,2,3,4}
  $$
* 외생변수
  $$
  \mathbf z_k = [T_{out}, RH, P_{high}, P_{low}, \dots]
  $$

**의미**
“지금 이 상태·입력·환경에서 운전을 계속하면, 각 실내기가 목표 온도에 도달하는 데 걸리는 시간”

---

## (2) 물리 변환: 뉴턴 냉각법칙 기반 $\Phi$

딥러닝 출력은 시간이므로, 이를 **1-step 온도 업데이트**로 변환해야 함.

각 실내기 (i)에 대해:

$$
T_{i,k+1}
=

T_{eq,i}
+
\big(T_{i,k}-T_{eq,i}\big)
\exp!\left(-\frac{\Delta t}{\tau_{i,k}}\right)
$$

* 평형 온도
  $$
  T_{eq,i} = T_{set,i} = T_{goal,i}-1
  $$
* $(\tau_{i,k})$:
  $$
  \tau_{i,k}
  = \Psi\big(\hat t_i,,T_{i,k},,T_{eq,i}\big)
  $$
(도달시간 → 시정수 역산 함수)

이로부터 **합성 1-step 모델**

$$
\mathbf x_{k+1}^{DL}
=

f(\mathbf x_k,\mathbf u_k,\mathbf z_k)

= \Phi\left(\mathbf x_k,\ g(\mathbf x_k,\mathbf u_k,\mathbf z_k)\right)
$$

---

# 2. 초기 제어 단계 (Planning Phase)

## 목적

* 아직 SSM이 없거나 신뢰도가 낮은 구간
* **딥러닝 모델을 직접 써서 전역적으로 좋은 입력을 찾음**

### 목적함수 예시

$$
J(\mathbf u)
=

w_1,\mathrm{Var}(\hat t_1,\hat t_2,\hat t_3)
+
w_2,\max_i \hat t_i
+
w_3,E(\mathbf u)
$$

* 온도 불균형 최소화
* 가장 늦게 도달하는 실내기 최소화
* 에너지 패널티

이산 입력 공간 $\mathbf u\in{0,\dots,4}^3$에서 **초기 최적 입력 $\mathbf u_0^\star$** 결정

---

# 3. SSM 식별 단계 (Model Construction Phase)

초기 입력 $\mathbf u_0^\star$ 근방에서 **선형 근사 모델**을 구성.

---

## 3.1 SSM 구조 정의


$$
\boxed{
\mathbf x_{k+1}
=

A_k \mathbf x_k
+
B_k \mathbf u_k
+
\mathbf d_k
}
$$

* $A_k$: 열 혼합 + 관성
* $B_k$: 풍량에 따른 냉방 효과
* $\mathbf d_k$:

  * 외생변수 영향
  * 모델이 학습한 비선형 잔차
  

---

## 3.2 (A,B) 식별 (주변 입력 기반 회귀)

풍량이 **이산(discrete)** 이므로 주변 입력/상태 샘플 기반 회귀

### 데이터 생성

기준점 $(\mathbf x_k,\mathbf u_0)$ 주변에서 M개의 입력, 상태 샘플링 ($\mathbf u_0$는 이전 시점에서 선택된 입력)

$$
{(\mathbf x_k^{(m)},\mathbf u_k^{(m)})}_{m=1}^M
$$

$\mathbf x^{(m)}$는 실측된 현재온도, +-1도 혹은 $\mathbf x_{k-1}, \mathbf x_{k-2}$를 사용, $\mathbf u^{(m)}$는 이전 입력, +-1단계 사용

각 실내기별로 상태와 입력 3차원이니까 경우의 수는 $3^3$ x $3^3$ 이중에 현실적으로 10~20 개 정도만 샘플링

각 샘플에 대해:

$$
\mathbf y^{(m)} =\mathbf x_{k+1}^{DL,(m)}
=

f(\mathbf x_k^{(m)},\mathbf u_k^{(m)},\mathbf z_k)
$$

### 회귀식

$$
\mathbf y^{(m)}
\approx
A_k \mathbf x_k^{(m)}
+
B_k \mathbf u_k^{(m)}
$$

최소자승으로 $A_k,B_k$ 추정.

$$
\Delta \mathbf y^{(m)} := \mathbf y^{(m)}-\mathbf y_0
$$
$$
\Delta \mathbf x^{(m)} := \mathbf x^{(m)}-\mathbf x_0,\quad
\Delta \mathbf u^{(m)} := \mathbf u^{(m)}-\mathbf u_0
$$

그러면
$$
\boxed{\Delta \mathbf y^{(m)} \approx A,\Delta\mathbf x^{(m)} + B,\Delta\mathbf u^{(m)}}
$$

이걸 (M)개 쌓아서 한 번에 최소제곱으로 푼다.

#### 행렬로 쌓기

$$
\Delta Y = [\Delta \mathbf y^{(1)}\ \cdots\ \Delta \mathbf y^{(M)}]\in\mathbb{R}^{3\times M}
$$
$$
\Delta X = [\Delta \mathbf x^{(1)}\ \cdots\ \Delta \mathbf x^{(M)}]\in\mathbb{R}^{3\times M}
$$
$$
\Delta U = [\Delta \mathbf u^{(1)}\ \cdots\ \Delta \mathbf u^{(M)}]\in\mathbb{R}^{3\times M}
$$

$$
\Delta Y \approx [A\ \ B]\begin{bmatrix}\Delta X\ \Delta U\end{bmatrix}
$$

$[A\ \ B]$는 식별해야할 값이고 나머지는 데이터


#### 미지수 묶기

$$
\Theta := [A\ B] \in \mathbb{R}^{3\times 6}
$$

#### 입력 데이터 묶기

$$
\Delta Z :=
\begin{bmatrix}
\Delta X\
\Delta U
\end{bmatrix}
\in \mathbb{R}^{6\times M}
$$

그러면 식이 아래의 형태가 된다.

$$
\boxed{
\Delta Y \approx \Theta,\Delta Z
}
$$

---

#### 최소제곱

$$
\min_{\Theta};
|\Delta Y - \Theta,\Delta Z|_F^2
$$

모든 샘플에 대해 예측 오차 제곱합 최소화

---

#### 행렬 미분
목적함수:

$$
J(\Theta)=\mathrm{tr}\Big(
(\Delta Y-\Theta\Delta Z)
(\Delta Y-\Theta\Delta Z)^\top
\Big)
$$

이를 $\Theta$에 대해 미분해서 0으로 둔다.

#### 미분 결과

$$
\frac{\partial J}{\partial \Theta}
=

-2\Delta Y\Delta Z^\top
+
2\Theta(\Delta Z\Delta Z^\top)
$$

#### = 0 조건 (정규방정식, normal equation)

$$
\Theta(\Delta Z\Delta Z^\top)
=

\Delta Y\Delta Z^\top
$$

#### 양변 우측에 역행렬 곱하기

$$
\boxed{
\Theta
=

\Delta Y\Delta Z^\top
(\Delta Z\Delta Z^\top)^{-1}
}
$$

즉

$$
[A\ B]
\approx
\Delta Y\Delta Z^\top(\Delta Z\Delta Z^\top)^{-1}
$$


---
그런데 현실에서는 역행렬이 없는 싱귤러 행렬일 수 있다
$$
\Delta Z\Delta Z^\top \approx \text{ill-conditioned or singular}
$$

#### 안정화: ridge(티호노프) 정규화

$$
[A\ \ B] \approx \Delta Y , \Delta Z^\top \big(\Delta Z,\Delta Z^\top+\lambda I\big)^{-1}
$$







이렇게 기준점 근처에서 유효한 SSM의 A, B를 식별

(냉기 확산, 단일 공간 혼합 효과가 (A,B)에 반영됨)

---

## 모델의 비선형성 흡수하는 외란항

기준점에서:

$$
\mathbf d_k^{(0)}
=

\mathbf x_{k+1}^{DL}

- \big(A_k \mathbf x_k + B_k \mathbf u_k\big)
$$

#### 모델에 의한 ssm 형태
$$
\mathbf x_{k+1}
=

A_k \mathbf x_k
+
B_k \mathbf u_k
+
\mathbf d_k^{(0)}
$$
---

# 4. 실측 보정 단계 (Adaptive Disturbance Update)



> **“구조(A,B)는 믿고, 현실 변화는 외란 (d_k)로 흡수한다.”**

---

## 4.1 실측 기반 외란 관측

실제 온도 측정:

$$
\mathbf x_{k}^{meas}
$$

외란 관측치:

$$
\tilde{\mathbf d}_k
=

\mathbf x_{k+1}^{meas}

-\big(A_k \mathbf x_k + B_k \mathbf u_k\big)
$$

좋아요. 제안하신 두 외란 정의는 **구조적으로 아주 타당**하고, 실제로 “DL을 외란 프라이어(prior)”, “실측을 외란 관측(measurement)”로 두는 **최적 결합 형태**입니다.
다만 한 가지를 꼭 정리해야 합니다:

* (d_k^{DL})은 **(k\to k+1)** 스텝에 대한 “미리보기(prior)”
* (d_k^{meas})는 **(k-1\to k)** 스텝에 대한 “사후(측정)”

즉, **시간 인덱스가 서로 다른 스텝의 외란**이라 그대로 섞으면 타이밍이 한 칸 어긋날 수 있어요.
이를 해결하는 가장 깔끔한 방식은 “현재 스텝 외란 (d_k)”를 만들 때:

1. **지난 스텝 실측으로 업데이트된 값(사후)**을 먼저 만들고
2. 그걸 **현재 DL 프라이어로 예측(또는 보정)**해서 (d_k)로 쓰는 겁니다.

아래에 가장 깔끔한 루프를 수식으로 정리해드릴게요.

---

## 1) 당신 정의를 “시간적으로 맞게” 다시 쓰면

### (A) DL 기반: 다음 스텝 외란 프라이어

$$
\boxed{
d_k^{DL} = x_{k+1}^{DL} - (A_k x_k^{meas} + B_k u_k)
}
$$

* 의미: **지금 상태/입력에서 (k\to k+1)** 에서 기대되는 비선형/외생 영향(프라이어)

✅ 이건 완벽합니다.

### (B) 실측 기반: 지난 스텝 외란 관측

$$
\boxed{
\tilde d_{k-1}^{meas} = x_k^{meas} - (A_{k-1}x_{k-1}^{meas}+B_{k-1}u_{k-1})
}
$$

* 의미: **(k-1\to k)** 에 실제로 발생했던 외란(관측치)

✅ 이것도 정석입니다.
(사용자 식의 (d_k^{meas})는 사실 (\tilde d_{k-1}^{meas})가 더 정확한 인덱싱)

---

## 2) 두 개를 섞어 “현재 사용할 (d_k)”를 만드는 정석 결합

핵심은 **(d_k)는 (k\to k+1)에 쓰는 외란**이므로, 같은 스텝 기준으로 맞춰야 해요.

그래서 가장 간단한 가정 하나:

$$
d_k \approx d_{k-1} \quad (\text{외란은 한 스텝에서 크게 변하지 않는다})
$$

이를 이용하면, 방금 측정한 (\tilde d_{k-1}^{meas})를 “현재 외란의 베이스”로 가져올 수 있습니다.

### Step 1) 실측으로 외란 사후 업데이트 (지난 스텝에서 얻은 정보)

$$
\boxed{
d_k^{post} = (1-\alpha)d_{k-1}^{post} + \alpha,\tilde d_{k-1}^{meas}
}
$$

* 이게 “칼만필터처럼 부드럽게”에 해당하는 부분입니다(EMA = 고정이득 KF 느낌)

### Step 2) DL 프라이어와 결합해서 최종 (d_k) 생성

$$
\boxed{
d_k = (1-\beta),d_k^{post} + \beta,d_k^{DL}
}
$$

* (\beta): “DL 프라이어를 얼마나 믿을지”

  * (\beta) 작게: 실측 기반 부하 추정을 더 신뢰(현실추종)
  * (\beta) 크게: DL이 학습한 외생변수 비선형 보정을 더 신뢰(노이즈 억제)

그리고 이 (d_k)를 사용해서

$$
\hat x_{k+1} = A_k x_k^{meas} + B_k u_k + d_k
$$

로 MPC 평가/제어하면 됩니다.

---

## 3) 더 세련된 버전: “DL은 변화량만 제공” (실전에서 더 안정적인 경우가 많음)

DL 프라이어가 절대값으로 튈 수 있으니, **DL이 말해주는 ‘외란 변화’만 반영**하는 방식도 좋습니다.

DL이 제공하는 변화량:

$$
\Delta d_k^{DL} := d_k^{DL} - d_{k-1}^{DL}
$$

그럼

$$
\boxed{
d_k = d_k^{post} + \gamma,\Delta d_k^{DL}
}
$$

* 의미: 실측 기반 외란을 기본으로 두고,
* DL이 말해주는 “이번 스텝에는 외란이 이렇게 변할 것”만 살짝 반영

이 방식은 **바이어스는 실측이 잡고**, **빠른 환경 변화는 DL이 힌트**를 주는 형태라 제어가 안정적입니다.

---

## 4) 결론: 당신 제안은 “좋고”, 인덱스만 정리하면 바로 쓸 수 있음

* (d_k^{DL}): **(k\to k+1)** 프라이어 ✅
* (d_{k-1}^{meas}): **(k-1\to k)** 관측 ✅
* 두 개를 섞으려면 “같은 스텝 기준”으로 맞춰서

  * 실측으로 (d_k^{post}) 만들고
  * DL로 (d_k) 예측/보정하는 구조가 정석 ✅

---

원하시면, 위 결합을 **완전한 칼만필터 형태**로도 써드릴게요:

* 측정1: 실측 기반 (\tilde d_{k-1}^{meas}) (분산 (R_{meas}))
* 측정2: DL 기반 (d_k^{DL}) (분산 (R_{DL}))
* 그러면 (\beta) 같은 가중치가 자동으로 (R)에서 결정됩니다.

지금 단계에서는 “EMA + 가중결합”만으로도 충분히 연구/구현 모두 설득력 있게 갑니다.
